In [1]:
SparkContext

pyspark.context.SparkContext

In [2]:
!pwd

/home/almalinux/eda1-coursework/src/merizo_pipeline/merizo_pipeline


In [3]:
!pip3 install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://download.pytorch.org/whl/cpu/torch-2.0.1%2Bcpu-cp39-cp39-linux_x86_64.whl (195.4 MB)


In [4]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [6]:
sc.master

'yarn'

In [157]:
os.getcwd()

'/home/almalinux/eda1-coursework/src/merizo_pipeline/merizo_pipeline'

In [7]:
def read_dir(input_dir, output_dir):
    """
    Read file paths from HDFS using SparkContext.
    """
    # Use SparkContext to read files from HDFS
    # If files are located in a directory on HDFS, you can use textFile 
    file_rdd = sc.wholeTextFile(input_dir + "/AF-Q46839-F1-model_v4.pdb") 
    file_paths = file_rdd.collect()  # This retrieves the file paths as a list

    # Create a list of tuples with file path, id, and output directory
    return [(file_path, os.path.basename(file_path), output_dir) for file_path in file_paths]


In [14]:
input_dir = "/UP000000625_83333_ECOLI_v4/"

In [15]:
file_rdd = sc.wholeTextFiles(input_dir + "*.pdb")
file_paths_rdd = file_rdd.map(lambda x: (x[0], os.path.basename(x[0])))

In [16]:
file_paths_rdd.collect()

[('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A0A385XJ53-F1-model_v4.pdb',
  'AF-A0A385XJ53-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A0A385XJE6-F1-model_v4.pdb',
  'AF-A0A385XJE6-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A0A385XJK5-F1-model_v4.pdb',
  'AF-A0A385XJK5-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A0A385XJL2-F1-model_v4.pdb',
  'AF-A0A385XJL2-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A0A385XJL4-F1-model_v4.pdb',
  'AF-A0A385XJL4-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A0A385XJN2-F1-model_v4.pdb',
  'AF-A0A385XJN2-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A0A385XK32-F1-model_v4.pdb',
  'AF-A0A385XK32-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A5A605-F1-model_v4.pdb',
  'AF-A5A605-F1-model_v4.pdb'),
 ('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-A5

In [201]:
from subprocess import Popen, PIPE
from pyspark import SparkFiles
from tempfile import NamedTemporaryFile

def run_merizo_search(file_tuple):
    input_file_path, input_file_name = file_tuple
    print(f"File Name: {input_file_name}, File Path: {input_file_path}")
    print(f"RUNNINGGGG MERIZOO SEARCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH")
    # local_file_path = os.path.join('/tmp', input_file_name)
    # cmd = ['python3',
    #        '/home/almalinux/merizo_search/merizo_search/merizo.py',
    #        'easy-search',
    #        input_file_path,
    #        '/home/almalinux/data/cath-4.3-foldclassdb',
    #        input_file_name,
    #        'tmp',
    #        '--iterate',
    #        '--output_headers',
    #        '-d',
    #        'cpu',
    #        '--threads',
    #        '1'
    #       ]
    cmd = [
        'cat',
        input_file_path
    ]
    print(f'STEP 1: RUNNING MERIZO: {" ".join(cmd)}')
    p = Popen(cmd, stdin=PIPE,stdout=PIPE, stderr=PIPE)
    out, err = p.communicate()
    # Decode the byte output to string
    print("Output:")
    print(out.decode())  # Decode and print the standard output
    
    if err:
        print("Error:")
        print(err.decode())  # Decode and print the standard error

In [200]:
file_paths_rdd.map(run_merizo_search).collect()

File Name: AF-Q46839-F1-model_v4.pdb, File Path: hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb
RUNNINGGGG MERIZOO SEARCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
STEP 1: RUNNING MERIZO: python3 /home/almalinux/merizo_search/merizo_search/merizo.py easy-search hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb /home/almalinux/data/cath-4.3-foldclassdb AF-Q46839-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1
Output:69:>                                                         (0 + 1) / 1]

Error:
2024-12-06 16:38:56,794 | INFO | Starting easy-search with command: 

/home/almalinux/merizo_search/merizo_search/merizo.py easy-search hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb /home/almalinux/data/cath-4.3-foldclassdb AF-Q46839-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1

2024-12-06 16:38:59,230 | INFO | easy-search finished after segmentation. Segmentation of this PDB file was no

[None]

In [246]:
local_file_path = SparkFiles.get(input_dir + "/AF-Q46839-F1-model_v4.pdb")
local_file_path

'/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb'

In [186]:
!ls

IOStream.flush timed out
AF-Q46839-F1-model_v4.pdb_segment.tsv  pipeline.py
__init__.py			       pipeline_script.py
parser.py			       results_parser.py
pipeline_playground.ipynb


In [175]:
!pwd

IOStream.flush timed out
/home/almalinux/eda1-coursework/src/merizo_pipeline/merizo_pipeline


In [167]:
run_merizo_search(('hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb', 'AF-Q46839-F1-model_v4.pdb'))

File Name: AF-Q46839-F1-model_v4.pdb, File Path: hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb
RUNNINGGGG MERIZOO SEARCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
STEP 1: RUNNING MERIZO: python3 /home/almalinux/merizo_search/merizo_search/merizo.py easy-search hdfs://hostnode:9000/UP000000625_83333_ECOLI_v4/AF-Q46839-F1-model_v4.pdb /home/almalinux/data/cath-4.3-foldclassdb AF-Q46839-F1-model_v4.pdb tmp --iterate --output_headers -d cpu --threads 1


In [170]:
!ls

IOStream.flush timed out
__init__.py  pipeline_playground.ipynb	pipeline_script.py
parser.py    pipeline.py		results_parser.py


In [169]:
!rm -rf AF-Q46839-F1-model_v4.pdb AF-Q46839-F1-model_v4.pdb_segment.tsv 

IOStream.flush timed out


In [22]:
from io import BytesIO
file_rdd_3 = sc.binaryFiles(input_dir + "*.pdb")
file_content_rdd = file_rdd_3.map(lambda x: (os.path.basename(x[0]), x[1]))

In [23]:
file_content_rdd.collect()

24/12/06 17:35:33 ERROR Utils: Uncaught exception in thread task-result-getter-3
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newArray(Native Method)
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2134)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:489)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:447)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject(JavaSerializer.scala:87)
	at org.apache.spark.serializer.SerializerHelper$.deserializeFromChunkedBuffer(SerializerHelper.scala:52)
	at org.apache.spark.scheduler.DirectTaskResult.value(TaskResult.scala

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 9 tasks (1099.3 MiB) is bigger than spark.driver.maxResultSize (1024.0 MiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.immutable.List.foreach(List.scala:333)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1247)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1247)
	at scala.Option.foreach(Option.scala:437)
	at org.apache.spark.scheduler.DAGSch

[Stage 5:==========================================>                (5 + 1) / 7]

In [18]:
from subprocess import Popen, PIPE
from tempfile import NamedTemporaryFile

def run_merizo_search(file_tuple):
    input_file_name, file_content = file_tuple
    print(f"File Name: {input_file_name}")
    # local_file_path = os.path.join('/tmp', input_file_name)
    # Create a temporary file to hold the content
    with NamedTemporaryFile(delete=False, mode='wb') as temp_file:
        temp_file.write(file_content)
        temp_file_path = temp_file.name
    
    cmd = ['python3',
           '/home/almalinux/merizo_search/merizo_search/merizo.py',
           'easy-search',
           temp_file_path,
           '/home/almalinux/data/cath-4.3-foldclassdb',
           input_file_name,
           'tmp',
           '--iterate',
           '--output_headers',
           '-d',
           'cpu',
           '--threads',
           '1'
          ]

    print(f'STEP 1: RUNNING MERIZO: {" ".join(cmd)}')
    p = Popen(cmd, stdin=PIPE,stdout=PIPE, stderr=PIPE)
    out, err = p.communicate()
    # Decode the byte output to string
    print("Output:")
    print(out.decode())  # Decode and print the standard output
    
    if err:
        print("Error:")
        print(err.decode())  # Decode and print the standard error

In [19]:
file_content_rdd.map(run_merizo_search).collect()

[None]